In [1]:
!unzip data.zip

Archive:  data.zip
   creating: data/
   creating: data/test/
  inflating: data/test/Groupe1_Image1.jpg  
  inflating: data/test/Groupe1_Image4.jpg  
  inflating: data/test/Groupe1_Image5.jpg  
  inflating: data/test/Groupe1_Image9.jpg  
  inflating: data/test/Groupe2_Image1.jpeg  
  inflating: data/test/Groupe2_Image11.jpg  
  inflating: data/test/Groupe2_Image9.jpg  
  inflating: data/test/Groupe5_image04.jpg  
  inflating: data/test/Groupe5_image08.jpg  
  inflating: data/test/Groupe5_image11.jpeg  
  inflating: data/test/Groupe5_image13.jpeg  
  inflating: data/test/Groupe5_image17.jpeg  
  inflating: data/test/Groupe6_image6.jpg  
  inflating: data/test/img4.jpg      
  inflating: data/test/img6.jpg      
  inflating: data/test/t3i13.jpg     
  inflating: data/test/t3i14.jpeg    
  inflating: data/test/t3i20.png     
  inflating: data/test/t3i21.jpeg    
  inflating: data/test/t3i28.jpg     
   creating: data/train/
  inflating: data/train/Groupe1_Image10.jpg  
  inflating: data/t

In [5]:
import cv2
import numpy as np
import imutils
import json
import os
import pandas as pd

def find_optimal_canny_threshold(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) if len(image.shape) == 3 else image
    otsu_threshold, _ = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return int(otsu_threshold * 0.5), int(otsu_threshold * 1.5)

def preprocess_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) if len(image.shape) == 3 else image
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    sobel_x = cv2.Sobel(blurred, cv2.CV_16S, 1, 0, ksize=5)
    sobel_y = cv2.Sobel(blurred, cv2.CV_16S, 0, 1, ksize=5)
    sobel_combined = cv2.addWeighted(cv2.convertScaleAbs(sobel_x), 0.5, cv2.convertScaleAbs(sobel_y), 0.5, 0)
    _, binary = cv2.threshold(sobel_combined, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    return binary

def detect_dominant_angle(binary):
    lines = cv2.HoughLinesP(binary, 1, np.pi / 180, 50, minLineLength=150, maxLineGap=10)
    if lines is None:
        return 0
    angles = [np.arctan2(y2 - y1, x2 - x1) * 180 / np.pi for line in lines for x1, y1, x2, y2 in [line[0]]]
    return np.median(angles) if angles else 0

def rotate_image(image, angle):
    (h, w) = image.shape[:2]
    matrix = cv2.getRotationMatrix2D((w // 2, h // 2), -angle, 1.0)
    return cv2.warpAffine(image, matrix, (w, h))

def detect_stairs(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) if len(image.shape) == 3 else image
    low_threshold, high_threshold = find_optimal_canny_threshold(gray)
    edges = cv2.Canny(gray, low_threshold, high_threshold, apertureSize=5)
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, 50, minLineLength=100, maxLineGap=10)
    if lines is None:
        return 0, 0, 0
    detected_lines_y = sorted([(y1 + y2) // 2 for line in lines for x1, y1, x2, y2 in [line[0]]])
    merged_lines_y = []
    for y in detected_lines_y:
        if not merged_lines_y or abs(y - merged_lines_y[-1]) > 60:
            merged_lines_y.append(y)
    line_distances = np.diff(merged_lines_y) if len(merged_lines_y) > 1 else [0]
    return len(merged_lines_y), np.mean(line_distances) if line_distances.any() else 0, np.max(line_distances) if line_distances.any() else 0

def process_images(data_path, gt_path):
    with open(gt_path, 'r') as f:
        ground_truth = json.load(f)

    results = []
    for entry in ground_truth["data"]:
        image_path = os.path.join(data_path, entry["image"])
        if not os.path.exists(image_path):
            continue

        image = cv2.imread(image_path)
        if image is None:
            continue

        image = imutils.resize(image, width=400)
        binary = preprocess_image(image)
        angle = detect_dominant_angle(binary)
        rotated_image = rotate_image(image, -angle) if abs(angle) > 5 else image
        num_stairs, avg_dist, max_dist = detect_stairs(rotated_image)

        results.append({
            "image": entry["image"],
            "actual_count": entry["actual_count"],
            "predicted_count": num_stairs,
            "absolute_error": abs(entry["actual_count"] - num_stairs),
            "avg_step_distance": avg_dist,
            "max_step_distance": max_dist
        })

    df = pd.DataFrame(results)
    df.to_csv("features.csv", index=False)
    print("Fichier features.csv généré !")

# Exécution
process_images("data/train", "gt.json")


Fichier features.csv généré !


In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Charger les caractéristiques extraites
features_df = pd.read_csv("features.csv")

# Supprimer les colonnes non numériques
features_df = features_df.drop(columns=["image"])

# Séparer les features et la variable cible
X = features_df.drop(columns=["actual_count"])
y = features_df["actual_count"]

# Diviser les données en train et test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entraîner un RandomForestRegressor
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Prédictions sur le test set
y_pred = model.predict(X_test)

# Évaluation du modèle
mae = mean_absolute_error(y_test, y_pred)
print(f"Erreur absolue moyenne (MAE): {mae:.2f}")

# Sauvegarde du modèle
import joblib
joblib.dump(model, "stair_detector_model.pkl")
print("Modèle sauvegardé sous 'stair_detector_model.pkl'")


Erreur absolue moyenne (MAE): 1.30
Modèle sauvegardé sous 'stair_detector_model.pkl'


In [9]:
import joblib
import pandas as pd
import cv2
import imutils
import numpy as np
import json
import os
from sklearn.metrics import mean_absolute_error
# Charger le modèle entraîné
model = joblib.load("stair_detector_model.pkl")


def test_model(new_data_path, gt_path=None):
    results = []
    y_true = []
    y_pred = []

    # Charger le fichier de vérité terrain s'il existe
    ground_truth = None
    if gt_path and os.path.exists(gt_path):
        with open(gt_path, 'r') as f:
            ground_truth = json.load(f)

    for image_name in os.listdir(new_data_path):
        image_path = os.path.join(new_data_path, image_name)
        if not os.path.isfile(image_path):
            continue

        image = cv2.imread(image_path)
        if image is None:
            continue

        image = imutils.resize(image, width=400)
        binary = preprocess_image(image)
        angle = detect_dominant_angle(binary)
        rotated_image = rotate_image(image, -angle) if abs(angle) > 5 else image
        num_stairs, avg_dist, max_dist = detect_stairs(rotated_image)

        # Créer un DataFrame avec les bons noms de colonnes
        feature_data = pd.DataFrame([[num_stairs, avg_dist, max_dist]],
                                    columns=["num_stairs", "avg_step_distance", "max_step_distance"])

        # Vérifier que toutes les colonnes sont bien présentes dans le modèle
        missing_cols = set(X.columns) - set(feature_data.columns)
        for col in missing_cols:
            feature_data[col] = 0  # Ajouter les colonnes manquantes avec des valeurs par défaut

        # Réordonner les colonnes pour correspondre à l'entraînement
        feature_data = feature_data[X.columns]

        # Prédire le nombre d'escaliers avec le modèle
        predicted_count = model.predict(feature_data)[0]

        actual_count = None
        error = None
        if ground_truth:
            for entry in ground_truth["data"]:
                if entry["image"] == image_name:
                    actual_count = entry["actual_count"]
                    error = abs(actual_count - predicted_count) if actual_count is not None else None
                    break

        results.append({
            "image": image_name,
            "actual_count": actual_count,
            "predicted_count": predicted_count,
            "absolute_error": error
        })

        # Stocker pour calculer la MAE
        if actual_count is not None:
            y_true.append(actual_count)
            y_pred.append(predicted_count)

        print(f"Image: {image_name} | Prédit: {predicted_count:.0f} | Réel: {actual_count} | Erreur: {error}")

    # Sauvegarder les résultats
    with open("gt_results.json", "w") as f:
        json.dump({"data": results}, f, indent=4)

    print("✅ Test terminé ! Résultats enregistrés dans gt_results.json")

    # Calculer la MAE
    if y_true and y_pred:
        mae = mean_absolute_error(y_true, y_pred)
        print(f"\n📊 Erreur Absolue Moyenne (MAE) sur l'ensemble test : {mae:.2f}")

# Exécution du test
test_model("data/test", "gt.json")



Image: img6.jpg | Prédit: 6 | Réel: 11 | Erreur: 5.18
Image: t3i21.jpeg | Prédit: 4 | Réel: 14 | Erreur: 9.6
Image: Groupe1_Image4.jpg | Prédit: 5 | Réel: 12 | Erreur: 7.08
Image: Groupe5_image11.jpeg | Prédit: 6 | Réel: 13 | Erreur: 7.3
Image: Groupe5_image17.jpeg | Prédit: 5 | Réel: 13 | Erreur: 7.69
Image: Groupe1_Image1.jpg | Prédit: 6 | Réel: 11 | Erreur: 4.84
Image: img4.jpg | Prédit: 5 | Réel: 37 | Erreur: 31.79
Image: t3i20.png | Prédit: 5 | Réel: 2 | Erreur: 2.8600000000000003
Image: Groupe5_image13.jpeg | Prédit: 6 | Réel: 3 | Erreur: 2.88
Image: t3i14.jpeg | Prédit: 5 | Réel: 4 | Erreur: 0.7199999999999998
Image: Groupe5_image08.jpg | Prédit: 6 | Réel: 14 | Erreur: 8.219999999999999
Image: Groupe6_image6.jpg | Prédit: 5 | Réel: 9 | Erreur: 3.79
Image: Groupe2_Image11.jpg | Prédit: 6 | Réel: 9 | Erreur: 3.41
Image: Groupe1_Image9.jpg | Prédit: 6 | Réel: 9 | Erreur: 2.7
Image: t3i13.jpg | Prédit: 6 | Réel: 4 | Erreur: 2.0599999999999996
Image: Groupe1_Image5.jpg | Prédit: 6 | 